In [ ]:
# 1) Instala dependencias
!pip install -q pandas chardet openpyxl

# 2) Importa librerías
import pandas as pd, chardet, csv, re
from google.colab import files

# 3) Detecta codificación y delimitador
def detect_encoding_and_delim(path):
    raw = open(path, "rb").read()
    enc = chardet.detect(raw)["encoding"] or "utf-8"
    sample = raw.decode(enc, errors="replace")[:4096]
    delim = csv.Sniffer().sniff(sample,
                                delimiters=[",",";","\t","|"]).delimiter
    return enc, delim

# 4) Parsea números en formato europeo
def parse_euro(col: pd.Series) -> pd.Series:
    s = col.astype(str).str.strip()
    s = s.str.replace(r'\.(?=\d{3}\b)', '', regex=True)
    s = s.str.replace(',', '.', regex=False)
    return pd.to_numeric(s, errors="coerce")

# 5) Sube y lee el CSV
uploaded = files.upload()
path = next(iter(uploaded))
enc, delim = detect_encoding_and_delim(path)
df = pd.read_csv(path, sep=delim, encoding=enc, dtype=str, engine="python")

# 6) Filtrar sólo Ventanilla única
df_vu = df[df["Impuesto"].str.contains("ventanilla única", case=False, na=False)].copy()

# 7) Parsea Base imponible y Cuota IVA
df_vu["base"]  = parse_euro(df_vu["Base Imponible Recalculada con IVA de PAIS"])
df_vu["cuota"] = parse_euro(df_vu["Cuota del impuesto"])

# 8) Agrupar por la cadena exacta 'Impuesto'
grouped = (
    df_vu.groupby("Impuesto", as_index=False)
         .agg(base_imponible=("base","sum"),
              cuota_iva      =("cuota","sum"))
)

# 9) Extrae país, tasa e IVA (Normal/Reducido)
pat = (
    r"Ventanilla única e-commerce\.\s*"
    r"(?P<pais>[^\d%]+?)\s+"
    r"(?P<tasa>\d+(?:[.,]\d+)?)%\s*IVA\s*(?P<tipo>Normal|Reducido)"
)
ext = grouped["Impuesto"].str.extract(pat)
grouped = pd.concat([grouped, ext], axis=1)

# 10) Normaliza tasa a “xx,yy”
grouped["tasa"] = grouped["tasa"].str.replace(".", ",", regex=False)

# 11) Mapea nombre de país a ISO2
iso_map = {
    "Irlanda":"IE","Portugal":"PT","Francia":"FR","Holanda":"NL",
    "Alemania":"DE","Croacia":"HR","Italia":"IT","Bélgica":"BE",
    "Polonia":"PL","Suecia":"SE","Dinamarca":"DK","República Checa":"CZ",
    "Austria":"AT","Finlandia":"FI","Estonia":"EE","Grecia":"EL",
    "Eslovenia":"SI","Eslovaquia":"SK","Chipre":"CY","Malta":"MT",
    "Letonia":"LV","Luxemburgo":"LU","Hungría":"HU","Lithuania":"LT"
}
grouped["pais"] = grouped["pais"].str.strip().replace({"República Checa":"República Checa"})
grouped["country"] = grouped["pais"].map(iso_map)

# 12) Reordena y selecciona columnas
result = grouped[["country","tasa","tipo","base_imponible","cuota_iva"]]

# 13) Excluir España
result = result[result["country"] != "ES"].copy()

# 14) Quitar signo “-” de los sumatorios de IVA Normal
mask_norm = result["tipo"] == "Normal"
result.loc[mask_norm, ["base_imponible","cuota_iva"]] = result.loc[mask_norm,
    ["base_imponible","cuota_iva"]].abs()

# 15) Ordenar y resetear índice
result = result.sort_values(["country","tasa","tipo"]).reset_index(drop=True)

# 16) Mostrar y exportar
pd.set_option("display.float_format", "{:,.2f}".format)
print("\nSumatorio Ventanilla Única (sin ES, IVA Normal positivos):")
print(result.to_string(index=False))

result.to_excel("/content/sumatorio_vu_por_pais_iva.xlsx", index=False)
print("\n→ Excel guardado en /content/sumatorio_vu_por_pais_iva.xlsx")

Saving ExportedData (1).csv to ExportedData (1) (42).csv

Sumatorio Ventanilla Única (sin ES, IVA Normal positivos):
country tasa     tipo  base_imponible  cuota_iva
     AT   20   Normal        6,859.99   1,371.94
     BE   12 Reducido           10.17       1.22
     BE   21   Normal        5,602.77   1,176.58
     CY   19   Normal          704.82     133.90
     CZ   21   Normal        1,324.89     278.24
     DE   19   Normal       47,878.93   9,096.85
     DE    7 Reducido          458.24      32.10
     DK   25   Normal        9,368.48   2,342.15
     EE   22   Normal          309.57      68.11
     EL   24   Normal        1,912.37     458.96
     FI   24   Normal        8,262.35   1,982.93
     FR   10 Reducido          232.88      23.29
     FR   20   Normal      151,812.52  30,361.88
     HR   25   Normal          663.24     165.82
     HU   27   Normal          226.44      61.14
     IE 13,5 Reducido          132.73      17.91
     IE   23   Normal       20,118.57   4,627.29
 